In [106]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [44]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


Pregnancies — количество беременностей.

Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

BloodPressure — диастолическое артериальное давление (мм рт. ст.).

SkinThickness — толщина кожной складки трицепса (мм).

Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).

BMI — индекс массы тела ((вес в кг)/ (рост в м)**2)

DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

Age — возраст.

Outcome — наличие диабета (0 — нет, 1 — да).

In [45]:
diabetes.shape

(778, 10)

Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных.

In [70]:
dupl_columns = list(diabetes.columns)

diabetes = diabetes.drop_duplicates(subset=dupl_columns)
diabetes.shape[0]

768

найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны.

In [71]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [72]:
information_diabetes = diabetes.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_diabetes.shape[1]}')

Результирующее число признаков: 9


найти пропуски в данных с помощью метода isnull()
Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

In [73]:
information_diabetes.isnull().value_counts()


Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI    DiabetesPedigreeFunction  Age    Outcome
False        False    False          False          False    False  False                     False  False      768
dtype: int64

# Вариант 1.
Использование метода .replace()

In [74]:

information_diabetes_nan = information_diabetes

information_diabetes_nan[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = information_diabetes_nan[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.nan)
information_diabetes_nan

round(information_diabetes_nan.Insulin.isnull().mean(), 2)

0.49

## Варинат 2
Создайте функцию, которая возвращает np.nan, если её аргумент равен 0, в противном случае — возвращает аргумент. Воспользуйтесь методом apply() и примените созданную функцию для каждого перечисленного столбца.

In [ ]:
# def change_nan(arg):
#     if arg == 0:
#         return np.nan
#     else:
#         arg
        
# information_diabetes_nan = information_diabetes[['Glucose', 'BloodPressure',
#                                                  'SkinThickness', 'Insulin', 'BMI']].apply(change_nan, axis=1)
# information_diabetes_nan

Удалите из данных признаки, где число пропусков составляет более 30 %.

In [75]:
#создаем копию исходной таблицы
drop_diabetes = information_diabetes_nan.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = drop_diabetes.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_diabetes = drop_diabetes.dropna(thresh=thresh, axis=1)
#  Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?
drop_diabetes.shape[1]

8

Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. 

In [76]:
n = drop_diabetes.shape[1]
drop_diabetes = drop_diabetes.dropna(thresh=n-2, axis=0)

# Чему равно результирующее число записей в таблице?
drop_diabetes.shape[0]


761

В оставшихся записях замените пропуски на медиану

In [97]:

fill_diabetes = drop_diabetes.copy()
fill_diabetes.isnull().sum()
values = {'Glucose': fill_diabetes.Glucose.median(),
          'BloodPressure': fill_diabetes.BloodPressure.median(),
          'SkinThickness': fill_diabetes.SkinThickness.median(),
          'BMI': fill_diabetes.BMI.median()
}
fill_diabetes = fill_diabetes.fillna(values)
# Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.
fill_diabetes.SkinThickness.mean().round(1)

29.1

Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [90]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

In [95]:
outliers, _ = outliers_iqr_mod(fill_diabetes, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')

Число выбросов по методу Тьюки: 87


Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [92]:
def outliers_z_score_mod(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

In [99]:
outliers, _ = outliers_z_score_mod(fill_diabetes, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')

Число выбросов по методу z-отклонения: 4


Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.

In [102]:
outliers_class, _ = outliers_iqr_mod(fill_diabetes, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers_class.shape[0]}')

Число выбросов по методу Тьюки: 29


Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!)

In [105]:
outliers_log, _ = outliers_iqr_mod(fill_diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу Тьюки: {outliers_log.shape[0]}')

# Какова разница между двумя этими числами (вычтите из первого второе)?
print(outliers_class.shape[0] - outliers_log.shape[0])

Число выбросов по методу Тьюки: 0
29


# задание на самопроверку
Имеются две базы данных (два листа Excel-файла): база с ценами конкурентов (Data_Parsing) и внутренняя база компании (Data_Company).

В базе парсинга есть два id, однозначно определяющие товар: producer_id и producer_color.

В базе компании есть два аналогичных поля: item_id и color_id.

Нам известно, что коды в двух базах отличаются наличием набора служебных символов. В базе парсинга встречаются следующие символы: _, -, ~, \\, /.

Считать данные из Excel в DataFrame (Data_Parsing) и (Data_Company).

In [108]:
Data_Parsing = pd.read_excel('data/Data_TSUM.xlsx')
# Data_Company = pd.read_excel('data/Data_TSUM.xlsx', 1)

Data_Parsing

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

Подтянуть к базе парсинга данные из базы компании (item_id, color_id, current_price) и сформировать столбец разницы цен в % (цена конкурента к нашей цене).

Определить сильные отклонения от среднего в разности цен в пределах бренда-категории (то есть убрать случайные выбросы, сильно искажающие сравнение). Критерий — по вкусу, написать комментарий в коде.

Записать новый файл Excel с базой парсинга, приклееными к ней столбцами из пункта 2 и с учётом пункта 3 (можно добавить столбец outlier и проставить Yes для выбросов).